In [1]:
import torch
import numpy as np

In [2]:
BOARD_SIZE = 8

Environment closely follows OpenAI gym API. Currently can not be invoked with ```gym.make("env_id")```, though it should be easy to do.

In [3]:
from env import Env

env = Env()

In [4]:
import monte_carlo_tree

def state_to_board(state):
    board = np.zeros((5, 8, 8))
    for piece in state['pieces']:
        if piece['color'] == "Black":
            board[0, piece['x'], piece['y']] = 1
        else: 
            board[1, piece['x'], piece['y']] = 1
        board[2] = 1 if state['turn']['color'] == "Black" else 0

    return board


class CheckersNode(monte_carlo_tree.Node):
    """Go Game Tree Node"""

    def whites(self):
        """White pieces on board"""
        return state_to_board(self.state)[1]
    
    def blacks(self):
        """Black pieces on board"""
        return state_to_board(self.state)[0]

    def get_piece_by_id(self, piece_id):
        pieces = list(
            filter(
                lambda p: p['id'] == piece_id, self.state['pieces']
                )
            )
        )

        if len(pieces) > 0:
            return pieces[0]

        return next(
            filter(lambda p: p['id'] == piece_id, 
                self.state['pieces']
            ))

    def get_piece_by_coord(self, x, y):
        return next(
            filter(lambda p: p['x'] == x and p['y'] == y, 
                self.state['pieces']
            ))

    def current_player(self):
        """Return 1 if current player plays black, and -1 for whites"""
        if self.state['turn']['color'] == 'Black':
            return 1
        return 0
    
    def possible_actions(self, player=None):
        """List of possible next actions"""
        if player == None:
            player = self.current_player()

        coords = []
        for piece_id in range(0, 24):
            piece = self.get_piece_by_id(piece_id)
            
            for action in self.state['moveset'][piece_id]:
                coords.append((piece['x'], piece['y'], action[0], action[1]))

        actions = np.zeros((8, 8, 8, 8))
        for c in coords:
            actions[c] = 1

        return actions

    def possible_actions_list(self):
        actions = self.possible_actions()
        return np.argwhere(actions==1)

    def prepare_action(self, action):
        return action

    def evaluate(self, env):
        return self.reward

    def prepare_action(self, action):
        pa = {
            'piece': self.get_piece_by_coord(action[0], action[1]),
            'square': {'x': int(action[2]), 'y': int(action[3])}
        }
        return pa

    def prepare_state(self, player=None):
        """
        Prepare game state X from perspective of current player
        [
            [ 1 -1 -1 ]
            [ 1  0  0 ]
            [ 0  0 -1 ]
        ]

        
        Where  
            1:  current player
            -1: opposing player
            
        """

        if player == None:
            player = self.current_player()

        # take advantage of game symmetry        
        state = self.blacks() - self.whites() if player == 1 else self.whites() - self.blacks()

        return state

## Random Play Tree

This is very basic algorithm that plays the game by making random moves. Sometimes it reaches the end goal, but overall it supper inneficient.

In [5]:
rt = monte_carlo_tree.RandomPlayTree(env, CheckersNode, BOARD_SIZE)


def random_play():
    '''
    Play a game using random tree strategy
    '''
    return rt.simulate(rt.root_node)
    

def build_stats(playfunc, n_games=100):
    '''
    Play a number of random games and display result
    '''

    black_wins = 0
    white_wins = 0
    draws = 0
    moves = []
    
    for _ in range(n_games):
        m = playfunc()

        reward = m.reward

        if reward > 0:
            black_wins += 1
        elif reward < 0:
            white_wins += 1
        elif reward == 0.0:
            draws += 1
       
        moves.append(m.depth())
    
    print("Blacks: ", black_wins, "Whites: ", white_wins, "Draws: ", draws, "Moves mean:", np.mean(moves))

In [7]:
build_stats(random_play, 100)

StopIteration: 

## Monte Carlo Search Tree

In [7]:
mcst = monte_carlo_tree.MonteCarloPlayTree(env, CheckersNode, BOARD_SIZE)

def mtsc_play():
    '''
    Play a game using MonteCarloSearchTree
    '''
    
    return mcst.simulate(mcst.root_node)

In [8]:
# build_stats(mtsc_play, 100)

## Alpha Zero

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class ActorCritic(nn.Module):

    def __init__(self, board_size=BOARD_SIZE):
        super(ActorCritic, self).__init__()
        
        self.board_size = board_size
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 8192, kernel_size=3, padding=1)
        self.layer1 = nn.Linear(8192, 4096)
        self.layer2 = nn.Linear(8192, 1)
        
    def forward(self, x):

        x = x.unsqueeze(1)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, 2)
        x = F.dropout(x, p=0.2, training=self.training)
        x = x.view(-1, 8192)
        
        prob = F.hardsigmoid(self.layer1(x))
        value = F.hardtanh(self.layer2(x))

        return prob.view(-1, 8, 8, 8, 8), value.view(-1, 1)

actor_critic_network = ActorCritic().to(device)

In [10]:
azt = monte_carlo_tree.GuidedMonteCarloPlayTree(env, CheckersNode, BOARD_SIZE, actor_critic_network, device)

In [11]:
losses = []

In [12]:
for loss in azt.train(200):
    losses.append(loss)
torch.save(actor_critic_network.state_dict(), "./checkers.pt")

Iteration # 0
number of actions:  96
Loss: tensor(271.6643, device='cuda:0', grad_fn=<SumBackward0>)
Iteration # 1
number of actions:  82
Loss: tensor(312.7006, device='cuda:0', grad_fn=<SumBackward0>)
Iteration # 2
number of actions:  90
Loss: tensor(167.1609, device='cuda:0', grad_fn=<SumBackward0>)
Iteration # 3
number of actions:  98
Loss: tensor(276.8374, device='cuda:0', grad_fn=<SumBackward0>)
Iteration # 4
number of actions:  96
Loss: tensor(365.6924, device='cuda:0', grad_fn=<SumBackward0>)
Iteration # 5
number of actions:  81
Loss: tensor(330.5529, device='cuda:0', grad_fn=<SumBackward0>)
Iteration # 6
number of actions:  91
Loss: tensor(340.1009, device='cuda:0', grad_fn=<SumBackward0>)
Iteration # 7
number of actions:  101
Loss: tensor(317.9117, device='cuda:0', grad_fn=<SumBackward0>)
Iteration # 8
number of actions:  96
Loss: tensor(239.1516, device='cuda:0', grad_fn=<SumBackward0>)
Iteration # 9
number of actions:  68
Loss: tensor(198.1733, device='cuda:0', grad_fn=<SumB

In [ ]:
from matplotlib import pyplot as plt 
plt.plot([l.detach().cpu().numpy() for l in losses])
plt.show()